In [ ]:
# Setup
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core.policy_builder import PolicyBuilder, PolicyMechanic
from core.combined_outlook import CombinedFiscalOutlookModel
from core.policies import PolicyType

print("✅ Custom policy tools loaded!")

---

## 1. Policy Builder Framework

PoliSim's PolicyBuilder lets you create custom scenarios:

```python
policy = PolicyBuilder(name="My Policy")
policy.add_mechanic(type, parameters)
policy.validate()
policy.simulate(years)
```

### Mechanic Types

| Type | Description | Example |
|------|-------------|----------|
| `revenue` | Tax/revenue changes | Wealth tax |
| `spending` | Expenditure changes | Defense cuts |
| `transfer` | Benefit programs | UBI |
| `regulatory` | Cost/behavior changes | Drug pricing |
| `structural` | System redesign | M4A |

In [ ]:
# Create a custom policy builder
class CustomPolicyBuilder:
    """Build custom policy configurations."""
    
    def __init__(self, name: str, description: str = ""):
        self.name = name
        self.description = description
        self.mechanics = []
        self.revenue_impacts = []  # (year, amount) tuples
        self.spending_impacts = []  # (year, amount) tuples
        self.assumptions = {}
    
    def add_revenue_change(self, 
                          mechanism: str,
                          annual_impact_bn: float,
                          start_year: int = 2025,
                          phase_in_years: int = 1):
        """Add revenue-changing mechanism."""
        self.mechanics.append({
            'type': 'revenue',
            'mechanism': mechanism,
            'annual_impact': annual_impact_bn * 1e9,
            'start_year': start_year,
            'phase_in': phase_in_years
        })
        print(f"✅ Added revenue: {mechanism} (${annual_impact_bn:+.0f}B/year)")
        return self
    
    def add_spending_change(self,
                           program: str,
                           annual_impact_bn: float,
                           start_year: int = 2025,
                           phase_in_years: int = 1):
        """Add spending-changing mechanism."""
        self.mechanics.append({
            'type': 'spending',
            'program': program,
            'annual_impact': annual_impact_bn * 1e9,
            'start_year': start_year,
            'phase_in': phase_in_years
        })
        print(f"✅ Added spending: {program} (${annual_impact_bn:+.0f}B/year)")
        return self
    
    def add_transfer_program(self,
                            program_name: str,
                            per_capita_annual: float,
                            eligible_population_m: float,
                            start_year: int = 2025):
        """Add transfer/benefit program."""
        total_cost = per_capita_annual * eligible_population_m * 1e6
        self.mechanics.append({
            'type': 'transfer',
            'program': program_name,
            'per_capita': per_capita_annual,
            'eligible_pop': eligible_population_m * 1e6,
            'total_cost': total_cost,
            'start_year': start_year
        })
        print(f"✅ Added transfer: {program_name} (${total_cost/1e9:.0f}B/year)")
        return self
    
    def summary(self):
        """Display policy summary."""
        print(f"\n📋 Policy: {self.name}")
        print("=" * 50)
        if self.description:
            print(f"Description: {self.description}")
        
        total_revenue = 0
        total_spending = 0
        
        print(f"\nMechanics ({len(self.mechanics)} total):")
        for m in self.mechanics:
            if m['type'] == 'revenue':
                total_revenue += m['annual_impact']
                print(f"   💰 {m['mechanism']}: ${m['annual_impact']/1e9:+.0f}B/year")
            elif m['type'] == 'spending':
                total_spending += m['annual_impact']
                print(f"   📤 {m['program']}: ${m['annual_impact']/1e9:+.0f}B/year")
            elif m['type'] == 'transfer':
                total_spending += m['total_cost']
                print(f"   🤝 {m['program']}: ${m['total_cost']/1e9:.0f}B/year")
        
        net_impact = total_revenue - total_spending
        print(f"\n📊 Net Annual Impact: ${net_impact/1e9:+.0f}B")
        print(f"   (Positive = deficit reduction, Negative = deficit increase)")
        return net_impact

print("✅ CustomPolicyBuilder class ready!")
print("\n📋 Methods:")
print("   • add_revenue_change()   - Tax/revenue mechanisms")
print("   • add_spending_change()  - Program spending changes")
print("   • add_transfer_program() - Benefit/transfer programs")
print("   • summary()              - Display policy overview")

---

## 2. Example: Balanced Budget Reform

Let's build a comprehensive deficit reduction policy:

In [ ]:
# Build a balanced budget reform
reform = CustomPolicyBuilder(
    name="Balanced Budget Reform 2025",
    description="Comprehensive deficit reduction through revenue and spending changes"
)

# Revenue increases
reform.add_revenue_change("Corporate Tax (21% → 28%)", +180)
reform.add_revenue_change("Capital Gains Parity", +120)
reform.add_revenue_change("Estate Tax Expansion", +50)
reform.add_revenue_change("Carbon Tax ($50/ton)", +200)

# Spending reductions
reform.add_spending_change("Defense (-10%)", -80)
reform.add_spending_change("Drug Price Negotiation", -100)
reform.add_spending_change("Administrative Efficiency", -50)
reform.add_spending_change("Subsidy Reform", -40)

# Show summary
net = reform.summary()

---

## 3. Example: Universal Basic Income

A more ambitious policy with transfers:

In [ ]:
# Build a UBI policy
ubi = CustomPolicyBuilder(
    name="Universal Basic Income",
    description="$1,000/month for all adults, funded by VAT and wealth tax"
)

# The transfer program
ubi.add_transfer_program(
    "UBI ($12,000/year)",
    per_capita_annual=12000,
    eligible_population_m=258  # Adult population
)

# Funding mechanisms
ubi.add_revenue_change("10% VAT", +800)
ubi.add_revenue_change("2% Wealth Tax (>$50M)", +300)
ubi.add_revenue_change("Financial Transaction Tax", +150)
ubi.add_revenue_change("Carbon Tax ($100/ton)", +400)

# Offset programs replaced by UBI
ubi.add_spending_change("SNAP Consolidation", -120)
ubi.add_spending_change("TANF Consolidation", -25)
ubi.add_spending_change("Housing Voucher Reduction", -30)

# Summary
net = ubi.summary()

---

## 4. Parameter Sensitivity Analysis

How do different assumptions affect outcomes?

In [ ]:
# Sensitivity analysis
def sensitivity_analysis(base_revenue_bn: float, 
                        base_spending_bn: float,
                        variations: list = [-0.2, -0.1, 0, 0.1, 0.2]):
    """
    Analyze how variations in estimates affect net impact.
    """
    results = []
    
    for rev_var in variations:
        for spend_var in variations:
            adj_rev = base_revenue_bn * (1 + rev_var)
            adj_spend = base_spending_bn * (1 + spend_var)
            net = adj_rev - adj_spend
            
            results.append({
                'rev_adj': f"{rev_var:+.0%}",
                'spend_adj': f"{spend_var:+.0%}",
                'net_impact': net
            })
    
    return results

# Run for balanced budget reform
base_rev = 550  # $B from our reform
base_spend = 270  # $B in spending changes

sensitivity = sensitivity_analysis(base_rev, base_spend)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 8))

variations = [-0.2, -0.1, 0, 0.1, 0.2]
matrix = np.zeros((len(variations), len(variations)))

for i, rev_var in enumerate(variations):
    for j, spend_var in enumerate(variations):
        adj_rev = base_rev * (1 + rev_var)
        adj_spend = base_spend * (1 + spend_var)
        matrix[i, j] = adj_rev - adj_spend

im = ax.imshow(matrix, cmap='RdYlGn', aspect='auto')

# Labels
ax.set_xticks(range(len(variations)))
ax.set_yticks(range(len(variations)))
ax.set_xticklabels([f"{v:+.0%}" for v in variations])
ax.set_yticklabels([f"{v:+.0%}" for v in variations])
ax.set_xlabel('Spending Adjustment')
ax.set_ylabel('Revenue Adjustment')
ax.set_title('Net Deficit Impact Sensitivity ($B/year)\nGreen = Deficit Reduction, Red = Increase')

# Add values to cells
for i in range(len(variations)):
    for j in range(len(variations)):
        text = ax.text(j, i, f"${matrix[i, j]:.0f}B",
                      ha="center", va="center", color="black", fontsize=9)

plt.colorbar(im, label='Net Impact ($B)')
plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   • Green cells = deficit reduction scenarios")
print("   • Red cells = deficit increase scenarios")
print("   • Center (0%, 0%) = base case estimate")

---

## 5. Running Custom Policy Simulation

In [ ]:
# Simulate our custom reform
def simulate_custom_policy(policy: CustomPolicyBuilder, years: int = 10):
    """Run simulation with custom policy adjustments."""
    
    # Get baseline
    model = CombinedFiscalOutlookModel()
    baseline = model.project_outlook(2024, 2024 + years)
    
    # Calculate policy impacts
    annual_impact = sum(
        m.get('annual_impact', 0) or -m.get('total_cost', 0)
        for m in policy.mechanics
    )
    
    # Adjust projections
    policy_debt = []
    policy_deficit = []
    cumulative_impact = 0
    
    for i, year in enumerate(range(2024, 2025 + years)):
        if year >= 2025:  # Policy starts 2025
            cumulative_impact += annual_impact
        
        policy_deficit.append(baseline['annual_deficit'][i] - annual_impact if year >= 2025 else baseline['annual_deficit'][i])
        policy_debt.append(baseline['total_debt'][i] - cumulative_impact)
    
    return {
        'years': list(range(2024, 2025 + years)),
        'baseline_debt': baseline['total_debt'],
        'baseline_deficit': baseline['annual_deficit'],
        'policy_debt': policy_debt,
        'policy_deficit': policy_deficit,
        'cumulative_savings': cumulative_impact
    }

# Run simulation
results = simulate_custom_policy(reform, years=10)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Debt comparison
ax1 = axes[0]
ax1.plot(results['years'], [d/1e12 for d in results['baseline_debt']], 
         'b-', label='Baseline', linewidth=2)
ax1.plot(results['years'], [d/1e12 for d in results['policy_debt']], 
         'g-', label='With Reform', linewidth=2)
ax1.fill_between(results['years'],
                [d/1e12 for d in results['baseline_debt']],
                [d/1e12 for d in results['policy_debt']],
                alpha=0.3, color='green', label='Savings')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Debt ($ Trillions)')
ax1.set_title('Debt Trajectory Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Deficit comparison
ax2 = axes[1]
ax2.bar([y - 0.2 for y in results['years']], 
        [d/1e9 for d in results['baseline_deficit']], 
        width=0.4, label='Baseline', color='blue', alpha=0.7)
ax2.bar([y + 0.2 for y in results['years']], 
        [d/1e9 for d in results['policy_deficit']], 
        width=0.4, label='With Reform', color='green', alpha=0.7)
ax2.set_xlabel('Year')
ax2.set_ylabel('Annual Deficit ($ Billions)')
ax2.set_title('Annual Deficit Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n📊 10-Year Summary:")
print(f"   • Cumulative debt reduction: ${results['cumulative_savings']/1e12:.2f}T")
print(f"   • Final year debt difference: ${(results['baseline_debt'][-1] - results['policy_debt'][-1])/1e12:.2f}T")

---

## 6. Your Turn: Design a Policy

Now create your own policy! Fill in the template below:

In [ ]:
# 🎯 YOUR CUSTOM POLICY
# Uncomment and modify the lines below

# my_policy = CustomPolicyBuilder(
#     name="Your Policy Name",
#     description="Brief description of your policy"
# )

# Revenue mechanisms (positive = more revenue)
# my_policy.add_revenue_change("Your Tax Change", +100)  # $100B/year

# Spending changes (negative = cuts, positive = increases)
# my_policy.add_spending_change("Your Spending Change", -50)  # $50B cut

# Transfer programs
# my_policy.add_transfer_program(
#     "Your Program",
#     per_capita_annual=5000,  # $/person/year
#     eligible_population_m=100  # Million people eligible
# )

# View summary
# my_policy.summary()

# Run simulation
# my_results = simulate_custom_policy(my_policy, years=10)

print("💡 Uncomment the code above to design your own policy!")
print("\n📚 Ideas to try:")
print("   • Green New Deal investment program")
print("   • Flat tax reform")
print("   • Social Security expansion")
print("   • Infrastructure package")
print("   • Defense modernization")

---

## 🎓 What You've Learned

✅ Build custom policies with revenue/spending/transfer mechanics  
✅ Sensitivity analysis shows how assumptions affect outcomes  
✅ Simulate custom policies against baseline projections  
✅ Visualize comparative impacts over time  

---

## 📚 Next Step

| Notebook | Topic | Time |
|----------|-------|------|
| **10** | Capstone: Full Analysis | 2-3 hours |

---

*Continue to Notebook 10: Capstone Project* →